In [21]:
from openai import OpenAI
from typing import Any
import IPython.display as ipd
from aimo2.utils import mdlatex
from copy import deepcopy
from transformers import AutoTokenizer

In [5]:
model = "casperhansen/deepseek-r1-distill-qwen-1.5b-awq"
client = OpenAI(api_key="-", base_url="http://localhost:8000/v1")
tokenizer = AutoTokenizer.from_pretrained(model)

In [97]:
history: Any = [
    {"role": "user", "content": "what is 1/2 + 1/3?"},
]
completion = client.chat.completions.create(
    model=model, messages=history, temperature=0.7
)
reply = completion.choices[0].message.content
history.append({"role": "assistant", "content": reply})
mdlatex(reply)

***\<think\>***

I need to add the fractions 1/2 and 1/3. To do this, I should find a common denominator. The least common denominator for 2 and 3 is 6.

Next, I'll convert each fraction to have this common denominator. 

1/2 can be multiplied by 3/3 to become 3/6.
1/3 can be multiplied by 2/2 to become 2/6.

Now, I can add the two fractions: 3/6 + 2/6 equals 5/6.

Therefore, the sum of 1/2 and 1/3 is 5/6.

***\</think\>***

To add the fractions $\frac{1}{2}$ and $\frac{1}{3}$, follow these steps:

1. **Find a Common Denominator:**
   
   The denominators are 2 and 3. The least common denominator (LCD) for these is 6.

2. **Convert Each Fraction to Have the Common Denominator:**
   
   - For $\frac{1}{2}$:
     $
     \frac{1}{2} = \frac{1 \times 3}{2 \times 3} = \frac{3}{6}
     $
     
   - For $\frac{1}{3}$:
     $
     \frac{1}{3} = \frac{1 \times 2}{3 \times 2} = \frac{2}{6}
     $

3. **Add the Fractions:**
   
   $
   \frac{3}{6} + \frac{2}{6} = \frac{5}{6}
   $

4. **Simplify if Possible:**
   
   The fraction $\frac{5}{6}$ is already in its simplest form.

**Final Answer:**
$
\boxed{\dfrac{5}{6}}
$

In [119]:
# in case the model does not make the final answer
# use hacks because </think> crashes the apply_chat_template
fake_history = deepcopy(history)
fake_history[-1]["content"] = (
    fake_history[-1]["content"].replace("</think>", "<think2>")
    + "\n\nI repeat, the final answer is: \\[\n\\boxed{"
)
prompt = tokenizer.apply_chat_template(
    fake_history,
    tokenize=False,
    add_generation_prompt=False,
    continue_final_message=True,
).replace("<think2>", "</think>")
completion = client.completions.create(
    model=model,
    prompt=prompt,
    temperature=0.6,
    # max_tokens=10000,
)
reply = fake_history[-1]["content"] + completion.choices[0].text
# history.append({"assistant": reply})
mdlatex(reply)

***\<think\>***

I need to add the fractions 1/2 and 1/3. To do this, I should find a common denominator. The least common denominator for 2 and 3 is 6.

Next, I'll convert each fraction to have this common denominator. 

1/2 can be multiplied by 3/3 to become 3/6.
1/3 can be multiplied by 2/2 to become 2/6.

Now, I can add the two fractions: 3/6 + 2/6 equals 5/6.

Therefore, the sum of 1/2 and 1/3 is 5/6.
<think2>

To add the fractions $\frac{1}{2}$ and $\frac{1}{3}$, follow these steps:

1. **Find a Common Denominator:**
   
   The denominators are 2 and 3. The least common denominator (LCD) for these is 6.

2. **Convert Each Fraction to Have the Common Denominator:**
   
   - For $\frac{1}{2}$:
     $
     \frac{1}{2} = \frac{1 \times 3}{2 \times 3} = \frac{3}{6}
     $
     
   - For $\frac{1}{3}$:
     $
     \frac{1}{3} = \frac{1 \times 2}{3 \times 2} = \frac{2}{6}
     $

3. **Add the Fractions:**
   
   $
   \frac{3}{6} + \frac{2}{6} = \frac{5}{6}
   $

4. **Simplify if Possible:**
   
   The fraction $\frac{5}{6}$ is already in its simplest form.

**Final Answer:**
$
\boxed{\dfrac{5}{6}}
$

I repeat, the final answer is: $
\boxed{5/6}
$

In [100]:
history

[{'role': 'user', 'content': 'what is 1/2 + 1/3?'},
 {'role': 'assistant',
  'content': "<think>\nI need to add the fractions 1/2 and 1/3. To do this, I should find a common denominator. The least common denominator for 2 and 3 is 6.\n\nNext, I'll convert each fraction to have this common denominator. \n\n1/2 can be multiplied by 3/3 to become 3/6.\n1/3 can be multiplied by 2/2 to become 2/6.\n\nNow, I can add the two fractions: 3/6 + 2/6 equals 5/6.\n\nTherefore, the sum of 1/2 and 1/3 is 5/6.\n</think>\n\nTo add the fractions \\(\\frac{1}{2}\\) and \\(\\frac{1}{3}\\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators are 2 and 3. The least common denominator (LCD) for these is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   - For \\(\\frac{1}{2}\\):\n     \\[\n     \\frac{1}{2} = \\frac{1 \\times 3}{2 \\times 3} = \\frac{3}{6}\n     \\]\n     \n   - For \\(\\frac{1}{3}\\):\n     \\[\n     \\frac{1}{3} = \\frac{1 \\times 2}{

In [111]:
reply

"<think>\nI need to add the fractions 1/2 and 1/3. To do this, I should find a common denominator. The least common denominator for 2 and 3 is 6.\n\nNext, I'll convert each fraction to have this common denominator. \n\n1/2 can be multiplied by 3/3 to become 3/6.\n1/3 can be multiplied by 2/2 to become 2/6.\n\nNow, I can add the two fractions: 3/6 + 2/6 equals 5/6.\n\nTherefore, the sum of 1/2 and 1/3 is 5/6.\n<think2>\n\nTo add the fractions \\(\\frac{1}{2}\\) and \\(\\frac{1}{3}\\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators are 2 and 3. The least common denominator (LCD) for these is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   - For \\(\\frac{1}{2}\\):\n     \\[\n     \\frac{1}{2} = \\frac{1 \\times 3}{2 \\times 3} = \\frac{3}{6}\n     \\]\n     \n   - For \\(\\frac{1}{3}\\):\n     \\[\n     \\frac{1}{3} = \\frac{1 \\times 2}{3 \\times 2} = \\frac{2}{6}\n     \\]\n\n3. **Add the Fractions:**\n   \n   \\[\n   \\fr

In [112]:
tokenizer.tokenize("\\[\n\\boxed{\\dfrac{5}{6}}\n\\]")

['\\',
 '[Ċ',
 '\\',
 'boxed',
 '{\\',
 'd',
 'frac',
 '{',
 '5',
 '}{',
 '6',
 '}}Ċ',
 '\\',
 ']']

In [59]:
fake_history = deepcopy(history)
huh = tokenizer.apply_chat_template(
    [{"role": "assistant", "content": "what"}],
    tokenize=False,
    continue_final_message=True,
)
print(huh)
print(type(huh))
# tokenizer.decode(huh)

<｜begin▁of▁sentence｜><｜Assistant｜>what
<class 'str'>


In [27]:
fake_history

[{'role': 'user', 'content': 'what is 1/2 + 1/3?'},
 {'role': 'assistant',
  'content': '<think>\nTo add the fractions 1/2 and 1/3, I first need to find a common denominator. The least common denominator for 2 and 3 is 6.\n\nNext, I convert each fraction to have this common denominator. 1/2 becomes 3/6 and 1/3 becomes 2/6.\n\nFinally, I add the two fractions together: 3/6 + 2/6 equals 5/6.\n</think>\n\nTo find the sum of \\(\\frac{1}{2} + \\frac{1}{3}\\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators are 2 and 3. The least common denominator (LCD) for 2 and 3 is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   \\[\n   \\frac{1}{2} = \\frac{3}{6} \\quad \\text{and} \\quad \\frac{1}{3} = \\frac{2}{6}\n   \\]\n\n3. **Add the Fractions:**\n   \n   \\[\n   \\frac{3}{6} + \\frac{2}{6} = \\frac{5}{6}\n   \\]\n\n4. **Simplify if Possible:**\n   \n   In this case, \\(\\frac{5}{6}\\) cannot be simplified further.\n\n**Final Answer:*

In [ ]:
injected = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "what is 1/2 + 1/3?"},
        {
            "role": "assistant",
            "content": "<think>\nTo add the fractions 1/2 and 1/3, I first need to find a common denominator. The least common denominator for 2 and 3 is 6.\n\nNext, I convert each fraction to have this common denominator. 1/2 becomes 3/6 and 1/3 becomes 2/6.\n\nFinally, I add the two fractions together: 3/6 + 2/6 equals 5/6.\n</think>\n\nTo find the sum of \\(\\frac{1}{2} + \\frac{1}{3}\\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators are 2 and 3. The least common denominator (LCD) for 2 and 3 is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   \\[\n   \\frac{1}{2} = \\frac{3}{6} \\quad \\text{and} \\quad \\frac{1}{3} = \\frac{2}{6}\n   \\]\n\n3. **Add the Fractions:**\n   \n   \\[\n   \\frac{3}{6} + \\frac{2}{6} = \\frac{5}{6}\n   \\]\n\n4. **Simplify if Possible:**\n   \n   In this case, \\(\\frac{5}{6}\\) cannot be simplified further.\n\n**Final Answer:**\n\n\\[\n\\boxed{\\dfrac{5}{6}}\n\\]. I repeat, the final answer is \\boxed{".,
        },
    ],
    tokenize=False,
    # add_generation_prompt=True,
    continue_final_message=True,
)

ValueError: substring not found

In [52]:
tokenizer.tokenize("\\boxed{}")

['\\', 'boxed', '{}']

In [23]:
fake_history

[{'role': 'user', 'content': 'what is 1/2 + 1/3?'},
 {'role': 'assistant',
  'content': '<think>\nTo add the fractions 1/2 and 1/3, I first need to find a common denominator. The least common denominator for 2 and 3 is 6.\n\nNext, I convert each fraction to have this common denominator. 1/2 becomes 3/6 and 1/3 becomes 2/6.\n\nFinally, I add the two fractions together: 3/6 + 2/6 equals 5/6.\n</think>\n\nTo find the sum of \\(\\frac{1}{2} + \\frac{1}{3}\\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators are 2 and 3. The least common denominator (LCD) for 2 and 3 is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   \\[\n   \\frac{1}{2} = \\frac{3}{6} \\quad \\text{and} \\quad \\frac{1}{3} = \\frac{2}{6}\n   \\]\n\n3. **Add the Fractions:**\n   \n   \\[\n   \\frac{3}{6} + \\frac{2}{6} = \\frac{5}{6}\n   \\]\n\n4. **Simplify if Possible:**\n   \n   In this case, \\(\\frac{5}{6}\\) cannot be simplified further.\n\n**Final Answer:*

In [13]:
tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": "you are an expert mathematician",
        },
        {
            "role": "user",
            "content": "Calculate the sum of all primes under 100.",
        },
        {
            "role": "assistant",
            "content": """<think>""",
        },
    ],
    tokenize=False,
    # add_generation_prompt=True,
    continue_final_message=True,
)

'<｜begin▁of▁sentence｜>you are an expert mathematician<｜User｜>Calculate the sum of all primes under 100.<｜Assistant｜><think>'

In [42]:
ipd.Markdown("\<think\> $ \\frac{1}{2} $ \\</think\\>")

\<think\> $ \frac{1}{2} $ \</think\>